In [1]:
import analyticslib 
from analyticslib import DataSet

# Load Data
- Load data from previous Data Cleansing stage



In [2]:
!ls -lrt

total 864
-rw-------. 1 sa7d-5e452b87283280-4ab9c5f515a1 users 11505 Mar 10 14:37 myfile.py
drwx------. 2 sa7d-5e452b87283280-4ab9c5f515a1 users  4096 Mar 16 16:13 test.csv
-rw-------. 1 sa7d-5e452b87283280-4ab9c5f515a1 users 14856 Mar 26 20:58 analyticslib.py
drwx------. 2 sa7d-5e452b87283280-4ab9c5f515a1 users  4096 Mar 26 20:59 __pycache__
drwx------. 2 sa7d-5e452b87283280-4ab9c5f515a1 users  4096 Mar 30 09:54 cleaned_data.csv
drwx------. 2 sa7d-5e452b87283280-4ab9c5f515a1 users  4096 Mar 30 12:55 created_features_p.parquet.20190330
drwx------. 2 sa7d-5e452b87283280-4ab9c5f515a1 users  4096 Mar 30 13:11 cleaned_data_p.parquet
drwx------. 2 sa7d-5e452b87283280-4ab9c5f515a1 users  4096 Mar 30 14:22 created_features_p.parquet
drwx------. 2 sa7d-5e452b87283280-4ab9c5f515a1 users  4096 Mar 30 18:18 created_features_p_balanced.parquet
-rw-------. 1 sa7d-5e452b87283280-4ab9c5f515a1 users 87592 Mar 30 19:01 best_model_1.h5
-rw-------. 1 sa7d-5e452b87283280-4ab9c5f515a1 users 76360 Mar 30 20

In [3]:
#dataMain = DataSet(spark_context=sc, filename="created_features_p.parquet", file_storage="local_parquet", sampling_mode="ALL")
dataMain = DataSet(spark_context=sc, filename="created_features_p_balanced.parquet", file_storage="local_parquet", sampling_mode="ALL")


loading local parquet


In [4]:
dataMain.dataset.count()

11742

In [5]:
dataMain.dataset.groupBy("PolyPhen").count().show()

+-----------------+-----+
|         PolyPhen|count|
+-----------------+-----+
|           benign| 3861|
|possibly_damaging| 3932|
|probably_damaging| 3949|
+-----------------+-----+



# Prepare data for MLLib models
- At this point:
   - Attributes have been selected
   - Attribute vectors have been assembled
   - Data has been normalized
- Now we will define steps for training and evaluating a model

In [6]:
df = dataMain.dataset

splits = df.randomSplit([0.9, 0.1])
df_train = splits[0]
df_test = splits[1]

print("Dataset count: %d" % (df.count()) )

print("Dataset train count: %d" % (df_train.count()) )
df_train.groupBy("PolyPhen").count().show()

print("Dataset test count: %d" % (df_test.count()) )
df_test.groupBy("PolyPhen").count().show()


Dataset count: 11742
Dataset train count: 10581
+-----------------+-----+
|         PolyPhen|count|
+-----------------+-----+
|           benign| 3463|
|possibly_damaging| 3539|
|probably_damaging| 3536|
+-----------------+-----+

Dataset test count: 1204
+-----------------+-----+
|         PolyPhen|count|
+-----------------+-----+
|           benign|  400|
|possibly_damaging|  387|
|probably_damaging|  374|
+-----------------+-----+



## MLLib Models

In [7]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer

indexer = StringIndexer(inputCol="PolyPhen", outputCol="label")


vectorAssembler = VectorAssembler(inputCols=["cadd_scores_vec", "freqs_vec", "CLASS","STRAND","LoFtool","BLOSUM62_norm","ORIGIN_LOG2", "CHROMVec", "IMPACTVec", "CLNVCVec", "SIFTVec"], outputCol="features")
#vectorAssembler = VectorAssembler(inputCols=["cadd_scores_vec"], outputCol="features")

In [8]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=5)

#from pyspark.ml.classification import MultilayerPerceptronClassifier
#layers = [5, 5, 4]
#mlp_trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)


#from pyspark.ml.classification import GBTClassifier
#gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)

In [9]:
from pyspark.ml import Pipeline

pipeline_dt = Pipeline(stages=[indexer, vectorAssembler, dt])
pipeline_rf = Pipeline(stages=[indexer, vectorAssembler, rf])


In [10]:
#pipeline_mlp = Pipeline(stages=[indexer, vectorAssembler, mlp_trainer])


In [11]:
df_train.limit(2).toPandas()

,cadd_scores_vec,freqs_vec,CLASS,STRAND,LoFtool,BLOSUM62_norm,ORIGIN_LOG2,CHROMVec,IMPACTVec,CLNVCVec,SIFTVec,PolyPhen
0,"[0.001, -4.314148]","[0.0, 1e-05, 0.0]",0,-1,0.0243,[1.0],0.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0)",benign
1,"[0.001, -3.413448]","[0.0055, 0.00296, 0.0078]",0,-1,0.5310,[-1.0],0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0)",benign


In [12]:
model_dt = pipeline_dt.fit(df_train)
model_rf = pipeline_rf.fit(df_train)


In [13]:
#model_mlp = pipeline_mlp.fit(df_train)

In [14]:
prediction_dt = model_dt.transform(df_test)
prediction_rf = model_rf.transform(df_test)


In [15]:
#prediction_mlp = model_mlp.transform(df_train)


In [16]:
# Select example rows to display.
print("decision tree:")
prediction_dt.select("prediction", "label", "features").show(5)
print("random forest:")
prediction_rf.select("prediction", "label", "features").show(5)
#print("multi layer perceptron:")
#prediction_mlp.select("*").show(5)


decision tree:
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       2.0|  2.0|(46,[0,1,2,3,4,5,...|
|       2.0|  2.0|(46,[0,1,6,7,8,10...|
|       2.0|  2.0|(46,[0,1,2,4,6,7,...|
|       2.0|  2.0|(46,[0,1,6,7,8,21...|
|       2.0|  2.0|(46,[0,1,6,7,8,9,...|
+----------+-----+--------------------+
only showing top 5 rows

random forest:
+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       2.0|  2.0|(46,[0,1,2,3,4,5,...|
|       2.0|  2.0|(46,[0,1,6,7,8,10...|
|       2.0|  2.0|(46,[0,1,2,4,6,7,...|
|       2.0|  2.0|(46,[0,1,6,7,8,21...|
|       2.0|  2.0|(46,[0,1,6,7,8,9,...|
+----------+-----+--------------------+
only showing top 5 rows



In [17]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(prediction_dt)
print("DT - accuracy = %g " % (accuracy))



# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(prediction_rf)
print("RF - accuracy = %g " % (accuracy))




# Select (prediction, true label) and compute test error
#evaluator = MulticlassClassificationEvaluator(
#    labelCol="label", predictionCol="prediction", metricName="accuracy")
#accuracy = evaluator.evaluate(prediction_mlp)
#print("MLP - Test Error = %g " % (1.0 - accuracy))



DT - accuracy = 0.628738 
RF - accuracy = 0.638704 


In [18]:
prediction_rf.select("*").show()

+-----------------+--------------------+-----+------+-------+-------------+-----------+---------------+-------------+-------------+-------------+-----------------+-----+--------------------+--------------------+--------------------+----------+
|  cadd_scores_vec|           freqs_vec|CLASS|STRAND|LoFtool|BLOSUM62_norm|ORIGIN_LOG2|       CHROMVec|    IMPACTVec|     CLNVCVec|      SIFTVec|         PolyPhen|label|            features|       rawPrediction|         probability|prediction|
+-----------------+--------------------+-----+------+-------+-------------+-----------+---------------+-------------+-------------+-------------+-----------------+-----+--------------------+--------------------+--------------------+----------+
|[0.001,-2.114737]|[0.0012,0.00311,0...|    1|     1|  0.077|        [1.0]|        0.0|(23,[13],[1.0])|(3,[0],[1.0])|(6,[0],[1.0])|(4,[3],[1.0])|           benign|  2.0|(46,[0,1,2,3,4,5,...|[0.74635849170433...|[0.14927169834086...|       2.0|
|[0.001,-2.002075]|     

In [19]:
from sklearn.metrics import classification_report
import numpy as np


In [20]:
for df_result, model_type in [(prediction_dt, "Decision Tree"), (prediction_rf, "Random Forest")]:
    y_pred = df_result.select("prediction").collect()
    y_lab  = df_result.select("label").collect()

    yp = list( map( lambda r: r[0], y_pred ))
    yl = list( map( lambda r: r[0], y_lab ))

    print("Classification report for %s" % (model_type))
    print(classification_report(yl, yp))

Classification report for Decision Tree
             precision    recall  f1-score   support

        0.0       0.47      0.52      0.49       393
        1.0       0.67      0.61      0.64       413
        2.0       0.75      0.76      0.75       398

avg / total       0.63      0.63      0.63      1204

Classification report for Random Forest
             precision    recall  f1-score   support

        0.0       0.49      0.41      0.45       393
        1.0       0.65      0.77      0.70       413
        2.0       0.75      0.73      0.74       398

avg / total       0.63      0.64      0.63      1204



In [21]:
len(y_pred)

1204

## Prepare data for Deep NN model

In [22]:
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.sql import functions as F
from pyspark.sql import types as T

def dense_to_array(v):
  new_array = list([float(x) for x in v])
  return new_array

def sparse_to_array(v):
  v = DenseVector(v)
  new_array = list([float(x) for x in v])
  return new_array

dense_to_array_udf = F.udf(dense_to_array, T.ArrayType(T.FloatType()))

sparse_to_array_udf = F.udf(sparse_to_array, T.ArrayType(T.FloatType()))


In [23]:
df_keras_src_train = df_train
df_keras_src_test = df_test
print(df_train.count(), df_test.count())



10538 1204


In [24]:
#NOTE.- For some reason, the UDF fails unless the data set is brought to one single node (coalesce(1))

df_keras_x_train = df_keras_src_train.coalesce(1) \
    .withColumn("cadd_a", dense_to_array_udf('cadd_scores_vec')) \
    .withColumn("freqs_a", dense_to_array_udf('freqs_vec')) \
    .withColumn("CHROM_a", dense_to_array_udf('CHROMVec')) \
    .withColumn("IMPACT_a", dense_to_array_udf('IMPACTVec')) \
    .withColumn("CLNVCV_a", dense_to_array_udf('CLNVCVec')) \
    .withColumn("SIFT_a", dense_to_array_udf('SIFTVec')) \
    .withColumn("BLOSUM62_a", dense_to_array_udf('BLOSUM62_norm')) \
    .drop('cadd_scores_vec') \
    .drop('freqs_vec') \
    .drop('CHROMVec') \
    .drop('IMPACTVec') \
    .drop('CLNVCVec') \
    .drop('SIFTVec') \
    .select("*")

df_keras_x_test = df_keras_src_test.coalesce(1) \
    .withColumn("cadd_a", dense_to_array_udf('cadd_scores_vec')) \
    .withColumn("freqs_a", dense_to_array_udf('freqs_vec')) \
    .withColumn("CHROM_a", dense_to_array_udf('CHROMVec')) \
    .withColumn("IMPACT_a", dense_to_array_udf('IMPACTVec')) \
    .withColumn("CLNVCV_a", dense_to_array_udf('CLNVCVec')) \
    .withColumn("SIFT_a", dense_to_array_udf('SIFTVec')) \
    .withColumn("BLOSUM62_a", dense_to_array_udf('BLOSUM62_norm')) \
    .drop('cadd_scores_vec') \
    .drop('freqs_vec') \
    .drop('CHROMVec') \
    .drop('IMPACTVec') \
    .drop('CLNVCVec') \
    .drop('SIFTVec') \
    .select("*")


In [25]:
print(df_keras_x_train.count(), df_keras_x_test.count())



10538 1204


In [26]:
df_keras_x_train.printSchema()

root
 |-- CLASS: integer (nullable = true)
 |-- STRAND: integer (nullable = true)
 |-- LoFtool: double (nullable = true)
 |-- BLOSUM62_norm: vector (nullable = true)
 |-- ORIGIN_LOG2: double (nullable = true)
 |-- PolyPhen: string (nullable = true)
 |-- cadd_a: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- freqs_a: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- CHROM_a: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- IMPACT_a: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- CLNVCV_a: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- SIFT_a: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- BLOSUM62_a: array (nullable = true)
 |    |-- element: float (containsNull = true)



In [27]:
df_keras_prep_train = df_keras_x_train.select("CLASS", "STRAND", "LoFtool", "ORIGIN_LOG2", \
            "cadd_a", df_keras_x_train.cadd_a.getItem(0), df_keras_x_train.cadd_a.getItem(1), \
            df_keras_x_train.freqs_a.getItem(0), df_keras_x_train.freqs_a.getItem(1), df_keras_x_train.freqs_a.getItem(2), \
            df_keras_x_train.CHROM_a.getItem(0), df_keras_x_train.CHROM_a.getItem(1), df_keras_x_train.CHROM_a.getItem(2), df_keras_x_train.CHROM_a.getItem(3), df_keras_x_train.CHROM_a.getItem(4), df_keras_x_train.CHROM_a.getItem(5),  \
            df_keras_x_train.CHROM_a.getItem(6), df_keras_x_train.CHROM_a.getItem(7), df_keras_x_train.CHROM_a.getItem(8), df_keras_x_train.CHROM_a.getItem(9), df_keras_x_train.CHROM_a.getItem(10), df_keras_x_train.CHROM_a.getItem(11),  \
            df_keras_x_train.CHROM_a.getItem(12), df_keras_x_train.CHROM_a.getItem(13), df_keras_x_train.CHROM_a.getItem(14), df_keras_x_train.CHROM_a.getItem(15), df_keras_x_train.CHROM_a.getItem(16), df_keras_x_train.CHROM_a.getItem(17), \
            df_keras_x_train.CHROM_a.getItem(18), df_keras_x_train.CHROM_a.getItem(19), df_keras_x_train.CHROM_a.getItem(20), df_keras_x_train.CHROM_a.getItem(21), df_keras_x_train.CHROM_a.getItem(22), \
            df_keras_x_train.IMPACT_a.getItem(0), df_keras_x_train.IMPACT_a.getItem(1), df_keras_x_train.IMPACT_a.getItem(2),  \
            df_keras_x_train.CLNVCV_a.getItem(0), df_keras_x_train.CLNVCV_a.getItem(1), df_keras_x_train.CLNVCV_a.getItem(2), df_keras_x_train.CLNVCV_a.getItem(3), df_keras_x_train.CLNVCV_a.getItem(4), df_keras_x_train.CLNVCV_a.getItem(4),  \
            df_keras_x_train.SIFT_a.getItem(0), df_keras_x_train.SIFT_a.getItem(1), df_keras_x_train.SIFT_a.getItem(2), df_keras_x_train.SIFT_a.getItem(3),  \
            df_keras_x_train.BLOSUM62_a.getItem(0),
            "PolyPhen").drop("cadd_a").drop("freqs_a").drop("CHROM_a").drop("IMPACT_a").drop("CLNVCV_a").drop("SIFT_a").drop("BLOSUM62_norm")

df_keras_prep_test = df_keras_x_test.select("CLASS", "STRAND", "LoFtool", "ORIGIN_LOG2", \
            "cadd_a", df_keras_x_test.cadd_a.getItem(0), df_keras_x_test.cadd_a.getItem(1), \
            df_keras_x_test.freqs_a.getItem(0), df_keras_x_test.freqs_a.getItem(1), df_keras_x_test.freqs_a.getItem(2), \
            df_keras_x_test.CHROM_a.getItem(0), df_keras_x_test.CHROM_a.getItem(1), df_keras_x_test.CHROM_a.getItem(2), df_keras_x_test.CHROM_a.getItem(3), df_keras_x_test.CHROM_a.getItem(4), df_keras_x_test.CHROM_a.getItem(5),  \
            df_keras_x_test.CHROM_a.getItem(6), df_keras_x_test.CHROM_a.getItem(7), df_keras_x_test.CHROM_a.getItem(8), df_keras_x_test.CHROM_a.getItem(9), df_keras_x_test.CHROM_a.getItem(10), df_keras_x_test.CHROM_a.getItem(11),  \
            df_keras_x_test.CHROM_a.getItem(12), df_keras_x_test.CHROM_a.getItem(13), df_keras_x_test.CHROM_a.getItem(14), df_keras_x_test.CHROM_a.getItem(15), df_keras_x_test.CHROM_a.getItem(16), df_keras_x_test.CHROM_a.getItem(17), \
            df_keras_x_test.CHROM_a.getItem(18), df_keras_x_test.CHROM_a.getItem(19), df_keras_x_test.CHROM_a.getItem(20), df_keras_x_test.CHROM_a.getItem(21), df_keras_x_test.CHROM_a.getItem(22), \
            df_keras_x_test.IMPACT_a.getItem(0), df_keras_x_test.IMPACT_a.getItem(1), df_keras_x_test.IMPACT_a.getItem(2),  \
            df_keras_x_test.CLNVCV_a.getItem(0), df_keras_x_test.CLNVCV_a.getItem(1), df_keras_x_test.CLNVCV_a.getItem(2), df_keras_x_test.CLNVCV_a.getItem(3), df_keras_x_test.CLNVCV_a.getItem(4), df_keras_x_test.CLNVCV_a.getItem(4),  \
            df_keras_x_test.SIFT_a.getItem(0), df_keras_x_test.SIFT_a.getItem(1), df_keras_x_test.SIFT_a.getItem(2), df_keras_x_test.SIFT_a.getItem(3),  \
            df_keras_x_test.BLOSUM62_a.getItem(0),
            "PolyPhen").drop("cadd_a").drop("freqs_a").drop("CHROM_a").drop("IMPACT_a").drop("CLNVCV_a").drop("SIFT_a").drop("BLOSUM62_norm")
            

In [28]:
df_keras_prep_train.printSchema()

root
 |-- CLASS: integer (nullable = true)
 |-- STRAND: integer (nullable = true)
 |-- LoFtool: double (nullable = true)
 |-- ORIGIN_LOG2: double (nullable = true)
 |-- cadd_a[0]: float (nullable = true)
 |-- cadd_a[1]: float (nullable = true)
 |-- freqs_a[0]: float (nullable = true)
 |-- freqs_a[1]: float (nullable = true)
 |-- freqs_a[2]: float (nullable = true)
 |-- CHROM_a[0]: float (nullable = true)
 |-- CHROM_a[1]: float (nullable = true)
 |-- CHROM_a[2]: float (nullable = true)
 |-- CHROM_a[3]: float (nullable = true)
 |-- CHROM_a[4]: float (nullable = true)
 |-- CHROM_a[5]: float (nullable = true)
 |-- CHROM_a[6]: float (nullable = true)
 |-- CHROM_a[7]: float (nullable = true)
 |-- CHROM_a[8]: float (nullable = true)
 |-- CHROM_a[9]: float (nullable = true)
 |-- CHROM_a[10]: float (nullable = true)
 |-- CHROM_a[11]: float (nullable = true)
 |-- CHROM_a[12]: float (nullable = true)
 |-- CHROM_a[13]: float (nullable = true)
 |-- CHROM_a[14]: float (nullable = true)
 |-- CHROM_a[

In [29]:
df_keras_prep_train.groupBy("PolyPhen").count().show()

df_keras_prep_test.groupBy("PolyPhen").count().show()

+-----------------+-----+
|         PolyPhen|count|
+-----------------+-----+
|           benign| 3463|
|possibly_damaging| 3539|
|probably_damaging| 3536|
+-----------------+-----+

+-----------------+-----+
|         PolyPhen|count|
+-----------------+-----+
|           benign|  398|
|possibly_damaging|  393|
|probably_damaging|  413|
+-----------------+-----+



In [30]:
df_keras_prep_cat_train = df_keras_prep_train.select("*", F.when(df_keras_prep_train.PolyPhen == 'benign', 0) \
                     .when(df_keras_prep_train.PolyPhen == 'possibly_damaging', 1).when(df_keras_prep_train.PolyPhen == 'probably_damaging', 2).otherwise(-1).alias("PolyPhen_cat")) \
                    .drop("PolyPhen").select("*")

df_keras_prep_cat_test = df_keras_prep_test.select("*", F.when(df_keras_prep_test.PolyPhen == 'benign', 0) \
                     .when(df_keras_prep_test.PolyPhen == 'possibly_damaging', 1).when(df_keras_prep_test.PolyPhen == 'probably_damaging', 2).otherwise(-1).alias("PolyPhen_cat")) \
                    .drop("PolyPhen").select("*")



df_keras_prep_cat_train.groupBy("PolyPhen_cat").count().show()
df_keras_prep_cat_test.groupBy("PolyPhen_cat").count().show()


+------------+-----+
|PolyPhen_cat|count|
+------------+-----+
|           0| 3463|
|           1| 3539|
|           2| 3536|
+------------+-----+

+------------+-----+
|PolyPhen_cat|count|
+------------+-----+
|           0|  398|
|           1|  393|
|           2|  413|
+------------+-----+



In [31]:
df_keras_prep_cat_train.printSchema()

root
 |-- CLASS: integer (nullable = true)
 |-- STRAND: integer (nullable = true)
 |-- LoFtool: double (nullable = true)
 |-- ORIGIN_LOG2: double (nullable = true)
 |-- cadd_a[0]: float (nullable = true)
 |-- cadd_a[1]: float (nullable = true)
 |-- freqs_a[0]: float (nullable = true)
 |-- freqs_a[1]: float (nullable = true)
 |-- freqs_a[2]: float (nullable = true)
 |-- CHROM_a[0]: float (nullable = true)
 |-- CHROM_a[1]: float (nullable = true)
 |-- CHROM_a[2]: float (nullable = true)
 |-- CHROM_a[3]: float (nullable = true)
 |-- CHROM_a[4]: float (nullable = true)
 |-- CHROM_a[5]: float (nullable = true)
 |-- CHROM_a[6]: float (nullable = true)
 |-- CHROM_a[7]: float (nullable = true)
 |-- CHROM_a[8]: float (nullable = true)
 |-- CHROM_a[9]: float (nullable = true)
 |-- CHROM_a[10]: float (nullable = true)
 |-- CHROM_a[11]: float (nullable = true)
 |-- CHROM_a[12]: float (nullable = true)
 |-- CHROM_a[13]: float (nullable = true)
 |-- CHROM_a[14]: float (nullable = true)
 |-- CHROM_a[

In [32]:
df_train_keras = df_keras_prep_cat_train
df_test_keras = df_keras_prep_cat_test

## Keras Deep NN Model

In [33]:
import keras
print('Current:\t', keras.__version__)
print('Expected:\t 2.1.3')

Using TensorFlow backend.


Current:	 2.1.3
Expected:	 2.1.3


In [34]:
import tensorflow as tf
print('Current:\t', tf.__version__)
print('Expected:\t 1.5.0')

Current:	 1.5.0
Expected:	 1.5.0


In [35]:
#!pip install keras==2.1.3

In [36]:
#!pip install tensorflow==1.5.0

In [37]:
#some learners constantly reported 502 errors in Watson Studio. 
#This is due to the limited resources in the free tier and the heavy resource consumption of Keras.
#This is a workaround to limit resource consumption

from keras import backend as K

K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))

In [38]:
print(df_train_keras.count(), df_test_keras.count())


10538 1204


In [39]:
num_classes = 3

In [41]:
pandas_data = df_train_keras.toPandas()
x_train = np.array( pandas_data.iloc[:, :46] )
y_train_0 = np.array( pandas_data.iloc[:, 46] ) 
y_train = keras.utils.to_categorical(y_train_0, num_classes)


In [42]:
pandas_data = df_test_keras.toPandas()
x_test = np.array( pandas_data.iloc[:, :46] )
y_test_0 = np.array( pandas_data.iloc[:, 46] )
y_test = keras.utils.to_categorical(y_test_0, num_classes)

In [43]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.models import load_model

batch_size = 128
epochs = 1

input_width = 46


In [ ]:
best_optimizer = ''
best_layer_size = 0
best_accuracy = 0
best_activation_fist_layer = ''

print('Lay.size: \tOptimizer: \tActiv_L1: \tLoss: \tAccuracy:')

for layer_size in range(32, 80, 1):
    for optimizer in ['adagrad','adadelta','rmsprop']:
        for activation_fist_layer in ['sigmoid','tanh','relu']:
    
            kmodel = Sequential()
            kmodel.add(Dense(layer_size, activation=activation_fist_layer, input_shape=(input_width,)))
            kmodel.add(Dropout(0.1))
            kmodel.add(Dense(layer_size, activation = 'sigmoid'))
            kmodel.add(Dropout(0.1))
            kmodel.add(Dense(num_classes, activation='softmax'))

            #kmodel.summary()

            kmodel.compile(loss='categorical_crossentropy',
                          optimizer=optimizer,
                    metrics=['accuracy'])

            kmodel.fit(x_train, y_train, batch_size=batch_size,
                  epochs=epochs, validation_data=(x_test, y_test), verbose=0)

            score = kmodel.evaluate(x_test, y_test, verbose=0)

            print('%d \t %s \t %s \t %f \t %f' % (layer_size, optimizer[0:4], activation_fist_layer[0:4], score[0], score[1]))

            if score[1]>best_accuracy:
                best_optimizer = optimizer
                best_layer_size = layer_size
                best_activation_fist_layer = activation_fist_layer
                best_loss = score[0]
                best_accuracy = score[1]
                print("*")
                kmodel.save('best_model.h5')
            

print("\nBest results:")
print('Layer size: %d \tOptimizer: %s \tActivation_L1: %s \tLoss: %f \tAccuracy: %f' % (best_layer_size, best_optimizer, best_activation_fist_layer, best_loss, best_accuracy))



In [ ]:
#Layer size: 71 	Optimizer: adadelta 	Activation_L1: relu 	Accuracy: 0.757624
batch_size = 128
best_layer_size = 60
best_optimizer='adagrad'
best_activation_fist_layer='sigmoid'
epochs=10


print("\nNow retraining with best parameters...")
kmodel = Sequential()
kmodel.add(Dense(best_layer_size, activation=best_activation_fist_layer, input_shape=(input_width,)))
kmodel.add(Dropout(0.1))
#kmodel.add(Dense(best_layer_size, activation = 'sigmoid'))
#kmodel.add(Dropout(0.1))
kmodel.add(Dense(num_classes, activation='softmax'))

kmodel.compile(loss='categorical_crossentropy', optimizer=best_optimizer,
                     metrics=['accuracy'])

kmodel.fit(x_train, y_train, batch_size=batch_size,
              epochs=epochs, validation_data=(x_test, y_test), verbose=0)

kprediction = kmodel.predict(x_test)

print("Done predicting")


In [44]:
kmodel = load_model("best_model_iter_32_80.h5")

In [45]:
y_test_0

array([0, 0, 0, ..., 2, 2, 2])

In [46]:
yp = kmodel.predict_classes(x_test)
yl  = y_test_0

print("Classification report for NN")
print(classification_report(yl, yp))

Classification report for NN
             precision    recall  f1-score   support

          0       0.75      0.75      0.75       398
          1       0.49      0.46      0.47       393
          2       0.66      0.70      0.68       413

avg / total       0.63      0.64      0.64      1204



### Deploy the best model to Watson Machine Learning

In [47]:
!ls
#!mv best_model.h5 best_model_iter_32_80.h5

analyticslib.py		  cleaned_data.csv		       myfile.py
best_model_1.h5		  cleaned_data_p.parquet	       __pycache__
best_model_capstone.tgz   created_features_p_balanced.parquet  test.csv
best_model.h5		  created_features_p.parquet
best_model_iter_32_80.h5  created_features_p.parquet.20190330


In [48]:

#Best results:
#Layer size: 71 	Optimizer: adadelta 	Activation_L1: relu 	Accuracy: 0.757624


#Best results:
#Layer size: 63 	Optimizer: adagrad 	Activation_L1: tanh 	Accuracy: 0.756832

In [49]:
!tar -zcvf best_model_capstone.tgz best_model_iter_32_80.h5

best_model_iter_32_80.h5


In [50]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
2019-03-31 00:49:17,711 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.


In [51]:
wml_credentials = {
  "apikey": "VHXlht1bZAoI4b1a8FBZ2gw40ahwxRONHEg25_K680Wi",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:pm-20:us-south:a/d582e46f99874f968a56be7438ead24b:386d8470-8340-4c25-a848-e6a3e2c3dc3b::",
  "iam_apikey_name": "auto-generated-apikey-3df6a2a7-71e5-4d4c-94a6-e6aee0a7ea54",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/d582e46f99874f968a56be7438ead24b::serviceid:ServiceId-b9514503-3c54-445a-b1ee-74eb47e4ddff",
  "instance_id": "386d8470-8340-4c25-a848-e6a3e2c3dc3b",
  "password": "fe53b083-986c-42e2-b704-e30d615aa4f1",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "3df6a2a7-71e5-4d4c-94a6-e6aee0a7ea54"
}

In [52]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [53]:
import json

In [54]:
model_props = {
    client.repository.ModelMetaNames.AUTHOR_NAME: "FernandoLopez",
    client.repository.ModelMetaNames.AUTHOR_EMAIL: "superserious.supernatural@gmail.com",
    client.repository.ModelMetaNames.NAME: "KerasCapstone",
    client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow",
    client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.5",
    client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES: [{"name":"keras", "version":"2.1.3"}]    
}

In [55]:
published_model = client.repository.store_model(model="best_model_capstone.tgz", meta_props=model_props)

In [56]:
client.deployments.list()

------------------------------------  ------------------  ------  --------------  ------------------------  --------------  -------------
GUID                                  NAME                TYPE    STATE           CREATED                   FRAMEWORK       ARTIFACT TYPE
36ca412a-48a9-4041-9ace-1295061bec70  created_best_model  online  DEPLOY_SUCCESS  2019-03-31T03:27:49.860Z  tensorflow-1.5  model
------------------------------------  ------------------  ------  --------------  ------------------------  --------------  -------------


In [57]:
published_model_uid = client.repository.get_model_uid(published_model)

In [58]:
model_details = client.repository.get_details(published_model_uid)

In [59]:
model_details

{'entity': {'author': {'name': 'FernandoLopez'},
  'deployments': {'count': 0,
   'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/386d8470-8340-4c25-a848-e6a3e2c3dc3b/published_models/832f6252-a1cf-40f5-83ea-475d51696190/deployments'},
  'evaluation_metrics_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/386d8470-8340-4c25-a848-e6a3e2c3dc3b/published_models/832f6252-a1cf-40f5-83ea-475d51696190/evaluation_metrics',
  'feedback_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/386d8470-8340-4c25-a848-e6a3e2c3dc3b/published_models/832f6252-a1cf-40f5-83ea-475d51696190/feedback',
  'latest_version': {'created_at': '2019-03-31T05:49:17.904Z',
   'guid': '8372bac7-d889-4d46-98d5-a267fd2b5f1b',
   'url': 'https://us-south.ml.cloud.ibm.com/v3/ml_assets/models/832f6252-a1cf-40f5-83ea-475d51696190/versions/8372bac7-d889-4d46-98d5-a267fd2b5f1b'},
  'learning_configuration_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/386d8470-8340-4c25-a848-e6a3e2c3dc3b/pub

In [60]:
created_deployment = client.deployments.create(published_model_uid, name="created_best_model")



#######################################################################################

Synchronous deployment creation for uid: '832f6252-a1cf-40f5-83ea-475d51696190' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='0d9a4359-e6d6-45fd-b83e-12fc0befb4ab'
------------------------------------------------------------------------------------------------




### Test deployed model

In [61]:
scoring_endpoint = created_deployment['entity']['scoring_url']
print(scoring_endpoint)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/386d8470-8340-4c25-a848-e6a3e2c3dc3b/deployments/0d9a4359-e6d6-45fd-b83e-12fc0befb4ab/online


PolyPhen coded severities:
- 0 = benign
- 1 = possibly damaging
- 2 = probably damaging



In [74]:
x_score_1 = x_test[10].tolist()

In [75]:
scoring_payload = {'values': [x_score_1]}

print('This submission:')
x_score_1

This submission:


[0.0,
 -1.0,
 0.984,
 0.0,
 0.02800000086426735,
 -0.8826540112495422,
 0.01510000042617321,
 0.0,
 0.014999999664723873,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0]

In [78]:
print('Answer for this submission should be: ', np.argmax(y_test[10]))


Answer for this submission should be:  0


In [79]:
predictions = client.deployments.score(scoring_endpoint, scoring_payload)
print('...answer is:', predictions['values'][0][1])

...answer is: 0
